# Running the model

Setup a `feisty` integration.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import yaml

import feisty

## Configure testcase to run

TODO: this is using the first column (which I believe is shallowest); want to add deepest column as well.
That will require refactoring the yaml file and also changing test_ds.

In [3]:
testcase = feisty.driver.simulate_testcase("tanh_shelf", "cyclic")

In [4]:
with open("matlab_first_timestep.yaml") as f:
    baselines = yaml.safe_load(f)


def compare(baselines_dict, da, full_table=True, markdown_formatting=False):
    if markdown_formatting:
        print(f"| {baselines_dict['dimname']} | Matlab Value | Python Value | Rel Err |")
        print("| --- | --- | --- | --- |")
        seps = ["| ", " | ", " | ", " |"]
    else:
        seps = ["", ": ", ", ", ""]
    for var in da[baselines_dict["dimname"]].data:
        sel_dict = {baselines_dict["dimname"]: var}
        matlab_val = baselines_dict["values"][var]
        py_val = da.sel(sel_dict).data
        if matlab_val != 0:
            rel_err = (py_val - matlab_val) / matlab_val
            if rel_err < 0:
                rel_err = -rel_err
        else:
            if py_val == 0:
                rel_err = 0
            elif py_val > 0:
                rel_err = np.inf
            elif py_val < 0:
                rel_err = -np.inf
        if full_table or rel_err > 1e-2:
            print(
                f"{seps[0]}{var}{seps[1]}{matlab_val:10.4e}{seps[2]}"
                + f"{py_val:10.4e}{seps[2]}{rel_err:10.4e}{seps[-1]}"
            )

## Run the model


In [5]:
%%time

nsteps = 1
testcase.run(nsteps)
testcase.ds

CPU times: user 256 ms, sys: 6.11 ms, total: 262 ms
Wall time: 266 ms


<xarray.Dataset>
Dimensions:                    (X: 22, group: 9, time: 1, fish: 8, feeding_link: 19)
Coordinates:
  * X                          (X) float64 -0.5 -0.2381 0.02381 ... 4.738 5.0
  * group                      (group) <U12 'Sf' 'Sp' ... 'Ld' 'benthic_prey'
  * fish                       (fish) <U2 'Sf' 'Sp' 'Sd' 'Mf' ... 'Md' 'Lp' 'Ld'
  * feeding_link               (feeding_link) <U15 'Sf_Zoo' ... 'Ld_benthic_p...
    predator                   (feeding_link) <U2 'Sf' 'Sp' 'Sd' ... 'Ld' 'Ld'
    prey                       (feeding_link) <U12 'Zoo' ... 'benthic_prey'
Dimensions without coordinates: time
Data variables: (12/16)
    biomass                    (time, group, X) float64 9.999e-06 ... 0.000186
    T_habitat                  (time, fish, X) float64 18.2 18.2 ... 18.2 18.2
    ingestion_rate             (time, fish, X) float64 0.2152 ... 6.439e-07
    predation_flux             (time, fish, X) float64 5.725e-12 ... 0.0
    predation_rate             (time, fish, X) float64 5.725e-07 ... 0.0
    metabolism_rate            (time, fish, X) float64 0.04296 ... 0.00488
    ...                         ...
    recruitment_flux           (time, fish, X) float64 1.535e-09 ... 0.0
    fish_catch_rate            (time, fish, X) float64 0.0 0.0 ... 0.0008219
    encounter_rate_link        (time, feeding_link, X) float64 2.294 ... 0.0
    encounter_rate_total       (time, fish, X) float64 2.294 2.294 ... 6.439e-07
    consumption_rate_max_pred  (time, fish, X) float64 0.2375 0.2375 ... 0.01062
    consumption_rate_link      (time, feeding_link, X) float64 0.2152 ... 0.0

In [6]:
if nsteps > 1:
    test_ds = testcase.ds.isel(X=0, time=0)
    for group in test_ds.group.data:
        testcase.ds.biomass.sel(group=group).isel(X=0).plot()
    plt.ylim([1e-6, 1])
    plt.yscale("log")
    plt.legend(test_ds.group.data)
else:
    test_ds = testcase.ds.isel(X=0, time=0)

In [7]:
compare(
    baselines["encounter"],
    test_ds["encounter_rate_link"],
    markdown_formatting=True,
)

| feeding_link | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf_Zoo | 2.2885e+00 | 2.2936e+00 | 2.2411e-03 |
| Sp_Zoo | 2.2885e+00 | 2.2936e+00 | 2.2411e-03 |
| Sd_Zoo | 2.2885e+00 | 2.2936e+00 | 2.2411e-03 |
| Mf_Zoo | 2.9714e-01 | 2.9781e-01 | 2.2411e-03 |
| Mf_Sf | 1.9837e-06 | 1.9836e-06 | 3.9203e-05 |
| Mf_Sp | 1.9837e-06 | 1.9836e-06 | 3.9203e-05 |
| Mf_Sd | 1.9837e-06 | 1.9836e-06 | 3.9203e-05 |
| Mp_Zoo | 2.9714e-01 | 2.9781e-01 | 2.2411e-03 |
| Mp_Sf | 1.9837e-06 | 1.9836e-06 | 3.9203e-05 |
| Mp_Sp | 1.9837e-06 | 1.9836e-06 | 3.9203e-05 |
| Mp_Sd | 1.9837e-06 | 1.9836e-06 | 3.9203e-05 |
| Md_benthic_prey | 1.7232e-04 | 1.7232e-04 | 2.6443e-05 |
| Lp_Mf | 2.8618e-07 | 2.8617e-07 | 3.9203e-05 |
| Lp_Mp | 5.7237e-07 | 5.7235e-07 | 3.9203e-05 |
| Lp_Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld_Mf | 8.9267e-08 | 8.9590e-08 | 3.6110e-03 |
| Ld_Mp | 1.7853e-07 | 1.7918e-07 | 3.6110e-03 |
| Ld_Md | 2.3805e-07 | 2.3891e-07 | 3.6110e-03 |
| Ld_benthic_prey | 

In [8]:
compare(
    baselines["consumption"],
    test_ds["consumption_rate_link"],
    markdown_formatting=True,
)

| feeding_link | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf_Zoo | 2.1520e-01 | 2.1523e-01 | 1.7479e-04 |
| Sp_Zoo | 2.1520e-01 | 2.1523e-01 | 1.7479e-04 |
| Sd_Zoo | 2.1520e-01 | 2.1523e-01 | 1.7479e-04 |
| Mf_Zoo | 4.2967e-02 | 4.2980e-02 | 2.8994e-04 |
| Mf_Sf | 2.8684e-07 | 2.8627e-07 | 1.9860e-03 |
| Mf_Sp | 2.8684e-07 | 2.8627e-07 | 1.9860e-03 |
| Mf_Sd | 2.8684e-07 | 2.8627e-07 | 1.9860e-03 |
| Mp_Zoo | 4.2967e-02 | 4.2980e-02 | 2.8994e-04 |
| Mp_Sf | 2.8684e-07 | 2.8627e-07 | 1.9860e-03 |
| Mp_Sp | 2.8684e-07 | 2.8627e-07 | 1.9860e-03 |
| Mp_Sd | 2.8684e-07 | 2.8627e-07 | 1.9860e-03 |
| Md_benthic_prey | 1.7091e-04 | 1.7090e-04 | 2.6241e-05 |
| Lp_Mf | 2.8616e-07 | 2.8615e-07 | 3.9203e-05 |
| Lp_Mp | 5.7232e-07 | 5.7230e-07 | 3.9203e-05 |
| Lp_Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld_Mf | 8.8259e-08 | 8.8578e-08 | 3.6113e-03 |
| Ld_Mp | 1.7652e-07 | 1.7716e-07 | 3.6113e-03 |
| Ld_Md | 2.3536e-07 | 2.3621e-07 | 3.6113e-03 |
| Ld_benthic_prey | 

In [9]:
compare(
    baselines["ingestion"],
    test_ds["ingestion_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 2.1520e-01 | 2.1523e-01 | 1.7479e-04 |
| Sp | 2.1520e-01 | 2.1523e-01 | 1.7479e-04 |
| Sd | 2.1520e-01 | 2.1523e-01 | 1.7479e-04 |
| Mf | 4.2968e-02 | 4.2981e-02 | 2.8989e-04 |
| Mp | 4.2968e-02 | 4.2981e-02 | 2.8989e-04 |
| Md | 1.7091e-04 | 1.7090e-04 | 2.6241e-05 |
| Lp | 8.5849e-07 | 8.5845e-07 | 3.9203e-05 |
| Ld | 4.9891e-05 | 5.0070e-05 | 3.5868e-03 |


In [10]:
compare(
    baselines["predation"],
    test_ds["predation_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 5.7369e-07 | 5.7255e-07 | 1.9860e-03 |
| Sp | 5.7369e-07 | 5.7255e-07 | 1.9860e-03 |
| Sd | 5.7369e-07 | 5.7255e-07 | 1.9860e-03 |
| Mf | 3.7442e-07 | 3.7473e-07 | 8.2129e-04 |
| Mp | 7.4884e-07 | 7.4946e-07 | 8.2129e-04 |
| Md | 2.3536e-07 | 2.3621e-07 | 3.6113e-03 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [11]:
compare(
    baselines["mortality"],
    test_ds["mortality_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Sp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Sd | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Mf | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Mp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Md | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Lp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Ld | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |


In [12]:
compare(
    baselines["fishing"],
    test_ds["fish_catch_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mf | 8.2192e-04 | 8.2192e-04 | 1.3191e-16 |
| Mp | 8.2192e-05 | 8.2192e-05 | 0.0000e+00 |
| Md | 8.2192e-05 | 8.2192e-05 | 0.0000e+00 |
| Lp | 8.2192e-04 | 8.2192e-04 | 1.3191e-16 |
| Ld | 8.2192e-04 | 8.2192e-04 | 1.3191e-16 |


In [13]:
compare(
    baselines["avail_energy"],
    test_ds["energy_avail_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.0767e-01 | 1.0770e-01 | 2.6576e-04 |
| Sp | 1.0767e-01 | 1.0770e-01 | 2.6576e-04 |
| Sd | 1.0767e-01 | 1.0770e-01 | 2.6576e-04 |
| Mf | 1.5598e-02 | 1.5607e-02 | 6.0841e-04 |
| Mp | 1.5598e-02 | 1.5607e-02 | 6.0841e-04 |
| Md | -4.2548e-03 | -4.2548e-03 | 1.8670e-06 |
| Lp | -4.8798e-03 | -4.8795e-03 | 5.3205e-05 |
| Ld | -1.4488e-03 | -1.4560e-03 | 4.9355e-03 |


In [14]:
compare(
    baselines["growth"],
    test_ds["growth_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.0762e-01 | 1.0765e-01 | 2.6644e-04 |
| Sp | 1.0762e-01 | 1.0765e-01 | 2.6644e-04 |
| Sd | 1.0762e-01 | 1.0765e-01 | 2.6644e-04 |
| Mf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mp | 1.5276e-02 | 1.5367e-02 | 5.9405e-03 |
| Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [15]:
compare(
    baselines["reproduction"],
    test_ds["reproduction_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mf | 1.4534e-02 | 1.5352e-02 | 5.6278e-02 |
| Mp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [16]:
compare(
    baselines["recruitment"],
    test_ds["recruitment_flux"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.4534e-09 | 1.5352e-09 | 5.6278e-02 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mf | 1.0762e-06 | 1.0765e-06 | 2.6644e-04 |
| Mp | 1.0762e-06 | 1.0765e-06 | 2.6644e-04 |
| Md | 1.0762e-06 | 1.0765e-06 | 2.6644e-04 |
| Lp | 1.5276e-07 | 1.5367e-07 | 5.9405e-03 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [17]:
compare(
    baselines["biomass"],
    test_ds["biomass"],
    markdown_formatting=True,
)

| group | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 9.9993e-06 | 9.9993e-06 | 8.1225e-06 |
| Sp | 9.9978e-06 | 9.9978e-06 | 5.7341e-08 |
| Sd | 9.9978e-06 | 9.9978e-06 | 5.7341e-08 |
| Mf | 1.1076e-05 | 1.1068e-05 | 7.0402e-04 |
| Mp | 1.1076e-05 | 1.1075e-05 | 4.7476e-05 |
| Md | 1.1030e-05 | 1.1030e-05 | 2.6002e-05 |
| Lp | 1.0093e-05 | 1.0094e-05 | 9.0167e-05 |
| Ld | 9.9746e-06 | 9.9745e-06 | 7.1689e-06 |
| benthic_prey | 2.0985e-03 | 2.0985e-03 | 2.4576e-05 |
